# Dimensión fecha

### Librerías necesarias

In [103]:
from datetime import date
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import holidays
import yaml
import locale
locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')

'es_ES.UTF-8'

### Conexión con la base de datos

In [104]:
with open('../config.yml', 'r') as f:
  config = yaml.safe_load(f)
  config_oltp = config['fuente']
  config_olap = config['bodega']

url_oltp = (f"{config_oltp['drivername']}://{config_oltp['user']}:{config_oltp['password']}@{config_oltp['host']}:"f"{config_oltp['port']}/{config_oltp['dbname']}")
url_olap = (f"{config_olap['drivername']}://{config_olap['user']}:{config_olap['password']}@{config_olap['host']}:"f"{config_olap['port']}/{config_olap['dbname']}")
#print(url_oltp)
#print(url_olap)
oltp_conn = create_engine(url_oltp)
olap_conn = create_engine(url_olap)

### Módulo de extracción

In [105]:
table_estadosservicio = pd.read_sql_table('mensajeria_estadosservicio', oltp_conn, columns=['fecha', 'hora'])
table_novedadesservicio = pd.read_sql_table('mensajeria_novedadesservicio', oltp_conn, columns=['fecha_novedad'])

table_novedadesservicio['fecha'] = table_novedadesservicio['fecha_novedad'].dt.date
#table_novedadesservicio['hora'] = table_novedadesservicio['fecha_novedad'].dt.time # intercambiar cambia el formato
table_novedadesservicio['hora'] = table_novedadesservicio['fecha_novedad'].dt.strftime('%H:%M:%S') # intercambiar cambiar el formato
#table_novedadesservicio.drop(columns=['fecha_novedad'], inplace=True)

dimension_fecha = pd.concat([table_estadosservicio, table_novedadesservicio.drop(columns=['fecha_novedad'])]).reset_index(drop=True)
dimension_fecha['fecha'] = pd.to_datetime(dimension_fecha['fecha'])
dimension_fecha['id'] = dimension_fecha.index
dimension_fecha.set_index('id', inplace=True)

dimension_fecha.shape[0]


133610

### Módulo de transformación

In [106]:
dimension_fecha["día"] = dimension_fecha["fecha"].dt.day
dimension_fecha["mes"] = dimension_fecha["fecha"].dt.month
dimension_fecha["año"] = dimension_fecha["fecha"].dt.year
dimension_fecha["día_semana"] = dimension_fecha["fecha"].dt.weekday
dimension_fecha["trimestre"] = dimension_fecha["fecha"].dt.quarter

dimension_fecha.head()

,fecha,hora,día,mes,año,día_semana,trimestre
id,,,,,,,
0,2024-01-29,01:13:32,29,1,2024,0,1
1,2024-01-30,18:45:12,30,1,2024,1,1
2,2024-02-06,11:34:04,6,2,2024,1,1
3,2024-02-01,14:50:39,1,2,2024,3,1
4,2024-04-06,16:11:21,6,4,2024,5,2


In [107]:
dimension_fecha["día_del_año"] = dimension_fecha["fecha"].dt.day_of_year
#dimension_fecha["día_del_mes"] = dimension_fecha["fecha"].dt.days_in_month
#dimension_fecha["nombre_mes"] = dimension_fecha["fecha"].dt.month_name()
dimension_fecha["nombre_mes"] = dimension_fecha["fecha"].dt.strftime('%B')
dimension_fecha["nombre_día"] = dimension_fecha["fecha"].dt.day_name()
dimension_fecha['nombre_día'] = dimension_fecha["nombre_día"].replace({'Monday': 'lunes', 'Tuesday': 'martes', 'Wednesday': 'miércoles', 'Thursday': 'jueves', 'Friday': 'viernes', 'Saturday': 'sábado', 'Sunday': 'domingo'})
#dimension_fecha["str_fecha"] = dimension_fecha["fecha"].dt.strftime("%d/%m/%Y")

dimension_fecha.head()

,fecha,hora,día,mes,año,día_semana,trimestre,día_del_año,nombre_mes,nombre_día
id,,,,,,,,,,
0,2024-01-29,01:13:32,29,1,2024,0,1,29,enero,lunes
1,2024-01-30,18:45:12,30,1,2024,1,1,30,enero,martes
2,2024-02-06,11:34:04,6,2,2024,1,1,37,febrero,martes
3,2024-02-01,14:50:39,1,2,2024,3,1,32,febrero,jueves
4,2024-04-06,16:11:21,6,4,2024,5,2,97,abril,sábado


In [108]:
co_holidays = holidays.CO(language="es")
dimension_fecha["es_festivo"] = dimension_fecha["fecha"].apply(lambda x:  x in co_holidays)
dimension_fecha["festivo"] = dimension_fecha["fecha"].apply(lambda x: co_holidays.get(x))
dimension_fecha["fin_de_semana"] = dimension_fecha["día_semana"].apply(lambda x: x>4)
dimension_fecha['fecha'] = dimension_fecha['fecha'].dt.date
dimension_fecha.head()

,fecha,hora,día,mes,año,día_semana,trimestre,día_del_año,nombre_mes,nombre_día,es_festivo,festivo,fin_de_semana
id,,,,,,,,,,,,,
0,2024-01-29,01:13:32,29,1,2024,0,1,29,enero,lunes,False,None,False
1,2024-01-30,18:45:12,30,1,2024,1,1,30,enero,martes,False,None,False
2,2024-02-06,11:34:04,6,2,2024,1,1,37,febrero,martes,False,None,False
3,2024-02-01,14:50:39,1,2,2024,3,1,32,febrero,jueves,False,None,False
4,2024-04-06,16:11:21,6,4,2024,5,2,97,abril,sábado,False,None,True


In [109]:
dimension_fecha['fecha'] = pd.to_datetime(dimension_fecha['fecha']).dt.date
dimension_fecha['hora'] = pd.to_datetime(dimension_fecha['hora'].astype(str).str.split('.').str[0], format='%H:%M:%S').dt.time

# Revisar duplicados en dimension_fecha
duplicados_fecha = dimension_fecha.duplicated(subset=['fecha', 'hora']).sum()
print("Duplicados en dimension_fecha:", duplicados_fecha)
dimension_fecha.drop_duplicates(subset=['fecha', 'hora'], inplace=True)

Duplicados en dimension_fecha: 1599


### Módulo de carga a la bodega de datos

In [110]:
try: # Realizar una consulta sencilla para verificar la conexión
    # Ejecutar una consulta para obtener el nombre de las tablas
    tables = pd.read_sql("SELECT table_name FROM information_schema.tables WHERE table_schema = 'public';", olap_conn)
    #print(tables)
    print("Conexión establecida y consulta realizada con éxito.")
except Exception as e:
    print(f"Error al conectar a la base de datos: {e}")

Conexión establecida y consulta realizada con éxito.


In [111]:
try:
  dimension_fecha.to_sql('dim_fecha', olap_conn, if_exists='replace')
except Exception as e:
  print(f"Error al cargar datos: {e}")